In [1]:
import os

In [2]:
%pwd

'x:\\deep_learning\\text-summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'x:\\deep_learning\\text-summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [14]:
import os
from text_summarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk
import pandas as pd

In [17]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_dataset(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [20]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-08-03 10:53:34,838: INFO :common :yaml file: config\config.yaml loaded successfully]
[2024-08-03 10:53:34,842: INFO :common :yaml file: params.yaml loaded successfully]
[2024-08-03 10:53:34,843: INFO :common :created directory at: artifacts]
[2024-08-03 10:53:34,846: INFO :common :created directory at: artifacts/data_transformation]


Generating train split: 14732 examples [00:00, 37078.24 examples/s]
Generating validation split: 818 examples [00:00, 17148.79 examples/s]
Generating test split: 819 examples [00:00, 16362.30 examples/s]
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\harsh\anaconda3\envs\textS\Lib\site-packages\transformers\tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 57173.16 examples/s]
